In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.contrib.slim as slim  # TensorFlow-Slim
from sklearn.model_selection import train_test_split

from io import BytesIO
from zipfile import ZipFile
import requests
import os
import time
import math

# Visualizations will be shown in the notebook.
%matplotlib inline

In [2]:
# Constants
NUMBER_OF_CLASSES = 43
IMAGE_SIZE = 32
NUMBER_OF_CHANNELS = 3

EPOCHS = 40
BATCH_SIZE = 128
BATCH_SIZE_INF = 2048
VALIDATION_SIZE = 0.2

LEARNING_RATE = 5e-3  # 0.0005 for ADAM - 5e-3 for plain gradient descent
DROPOUT_KEEP = 0.25 #0.5

MODEL_SAVE_PATH = "model.ckpt"

RESTORE = False  # restore previous model, don't train?
RESUME = False  # resume training from previously trained model?

In [3]:
def download_if_not_existing(file_url):
    if not os.path.isfile("data/test.p") or not os.path.isfile("data/train.p"):
        url = requests.get(file_url)
        zipfile = ZipFile(BytesIO(url.content))
        zipfile.extractall(path="data")
        print("data files downloaded and extracted")
    else:
        print("data file exist")

In [4]:
# Load pickled data

# TODO: Fill this in based on where you saved the training and testing data
%time download_if_not_existing("https://d17h27t6h515a5.cloudfront.net/topher/2016/November/581faac4_traffic-signs-data/traffic-signs-data.zip")

training_file = "data/train.p"
testing_file = "data/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

data file exist
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 325 µs
X_train shape: (39209, 32, 32, 3)
X_test shape: (12630, 32, 32, 3)


In [5]:
### Replace each question mark with the appropriate value.

# TODO: Number of training examples
n_train = X_train.shape[0]

# TODO: Number of testing examples.
n_test = X_test.shape[0]

# TODO: What's the shape of an traffic sign image?
image_shape = X_train.shape[1:3]

# TODO: How many unique classes/labels there are in the dataset.
n_classes = np.unique(y_train).shape[0]

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 39209
Number of testing examples = 12630
Image data shape = (32, 32)
Number of classes = 43


In [9]:
# install tflearn and h5py with: `pip install tflearn` and `conda install h5py`
import tflearn
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

In [20]:
network = input_data(shape=[None, 32, 32, 3])
network = conv_2d(network, 32, 3, activation="relu")
network = max_pool_2d(network, 2)
network = conv_2d(network, 64, 3, activation="relu")
network = conv_2d(network, 64, 3, activation="relu")
network = max_pool_2d(network, 2)
network = fully_connected(network, 512, activation="relu")
network = dropout(network, 0.5)
network = fully_connected(network, 10, activation="softmax")

network = regression(network, optimizer="adam", loss="categorical_crossentropy", learning_rate=0.001)

In [22]:
# Train using classifier
model = tflearn.DNN(network, tensorboard_verbose=0)
model.fit(X_train, y_train, n_epoch=50, shuffle=True, validation_set=(X_test, y_test), show_metric=True, batch_size=32, run_id="cnn1")

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instructions for updating:
Please switch to tf.summary.merge.
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tag

ResourceExhaustedError: OOM when allocating tensor with shape[4096,512]
	 [[Node: FullyConnected_2/W/Initializer/truncated_normal/TruncatedNormal = TruncatedNormal[T=DT_INT32, _class=["loc:@FullyConnected_2/W"], dtype=DT_FLOAT, seed=0, seed2=0, _device="/job:localhost/replica:0/task:0/gpu:0"](FullyConnected_2/W/Initializer/truncated_normal/shape)]]

Caused by op 'FullyConnected_2/W/Initializer/truncated_normal/TruncatedNormal', defined at:
  File "/home/carnd/anaconda3/envs/CarND-Traffic-Sign-Classifier-Project/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/carnd/anaconda3/envs/CarND-Traffic-Sign-Classifier-Project/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/carnd/anaconda3/envs/CarND-Traffic-Sign-Classifier-Project/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/carnd/anaconda3/envs/CarND-Traffic-Sign-Classifier-Project/lib

In [ ]:
model.predict()